The deterministic model is trained based on golden labels(manually labeled), the model takes in one data sample statistics as input and predict the sample label. The input data abstains Context_Specific class. The general accuracy is `94.01`.

## After prepossessing

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.models import Model
from keras.layers import Dense, Embedding, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPool1D, Dropout, concatenate
from keras.preprocessing import text as keras_text, sequence as keras_seq
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(512)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from keras.layers import LeakyReLU
from keras import initializers
from keras.callbacks import ModelCheckpoint
from keras.callbacks import ReduceLROnPlateau
from keras.models import load_model
from keras.utils import to_categorical
import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold,StratifiedKFold
from keras.models import load_model

In [5]:
source = './../results/'

In [9]:
data = pd.read_csv(source+'processed_raw_4.csv')
# labels = np.loadtxt('train_marginals.txt')

In [10]:
data.head()

,Unnamed: 0,Unnamed: 0.1,y_act,Reason,Attribute_name,Total_val,num_of_dist_val,% nans,max_val,mean,...,sample_5,0,1,2,3,p_dist_val,p_nans,castability,extractability,len_val
0,0,0,3,l,StratificationCategory2,403984,1,0.803648,0.0,0.0,...,NaN,0.058316,0.433625,0.018591,0.489468,0.000002,0.803648,0,0,0.0
1,1,1,3,l,Stratification2,403984,1,0.803648,0.0,0.0,...,NaN,0.019360,0.451563,0.019360,0.509716,0.000002,0.803648,0,0,0.0
2,2,2,3,l,StratificationCategory3,403984,1,0.803648,0.0,0.0,...,NaN,0.058316,0.433625,0.018591,0.489468,0.000002,0.803648,0,0,0.0
3,3,3,3,l,Stratification3,403984,1,0.803648,0.0,0.0,...,NaN,0.019360,0.451563,0.019360,0.509716,0.000002,0.803648,0,0,0.0
4,4,4,2,c,GeoLocation,403984,55,0.006357,0.0,0.0,...,"(37.63864012300047, -120.99999953799971)",0.090072,0.028045,0.853837,0.028045,0.000136,0.006357,1,0,2.0


In [11]:
def abs_limit(x):
    if abs(x) > 10000:
        return 10000*np.sign(x)
    return x

In [12]:
data = data.fillna(0)
data['scaled_mean'] = data['mean'].apply(abs_limit)
data['scaled_min_val'] = data['min_val'].apply(abs_limit)
data['scaled_max_val'] = data['max_val'].apply(abs_limit)
data['scaled_std_dev'] = data['std_dev'].apply(abs_limit)

In [13]:
column_names_to_normalize = ['scaled_max_val', 'scaled_mean', 'scaled_min_val','scaled_std_dev']
x = data[column_names_to_normalize].values
x = np.nan_to_num(x)
x_scaled = StandardScaler().fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = data.index)
data[column_names_to_normalize] = df_temp

In [14]:
data.head()

,Unnamed: 0,Unnamed: 0.1,y_act,Reason,Attribute_name,Total_val,num_of_dist_val,% nans,max_val,mean,...,3,p_dist_val,p_nans,castability,extractability,len_val,scaled_mean,scaled_min_val,scaled_max_val,scaled_std_dev
0,0,0,3,l,StratificationCategory2,403984,1,0.803648,0.0,0.0,...,0.489468,0.000002,0.803648,0,0,0.0,-0.289384,0.032129,-0.41314,-0.351573
1,1,1,3,l,Stratification2,403984,1,0.803648,0.0,0.0,...,0.509716,0.000002,0.803648,0,0,0.0,-0.289384,0.032129,-0.41314,-0.351573
2,2,2,3,l,StratificationCategory3,403984,1,0.803648,0.0,0.0,...,0.489468,0.000002,0.803648,0,0,0.0,-0.289384,0.032129,-0.41314,-0.351573
3,3,3,3,l,Stratification3,403984,1,0.803648,0.0,0.0,...,0.509716,0.000002,0.803648,0,0,0.0,-0.289384,0.032129,-0.41314,-0.351573
4,4,4,2,c,GeoLocation,403984,55,0.006357,0.0,0.0,...,0.028045,0.000136,0.006357,1,0,2.0,-0.289384,0.032129,-0.41314,-0.351573


In [15]:
data.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'y_act', 'Reason', 'Attribute_name',
       'Total_val', 'num_of_dist_val', '% nans', 'max_val', 'mean', 'min_val',
       'std_dev', 'sample_1', 'sample_2', 'sample_3', 'sample_4', 'sample_5',
       '0', '1', '2', '3', 'p_dist_val', 'p_nans', 'castability',
       'extractability', 'len_val', 'scaled_mean', 'scaled_min_val',
       'scaled_max_val', 'scaled_std_dev'],
      dtype='object')

In [16]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=100)
data_train.shape, data_test.shape

((6952, 30), (1739, 30))

In [17]:
x_columns =  ['Attribute_name',
                  'sample_1',
                  'sample_2',
                  'sample_3',
                  'sample_4',
                  'sample_5',
                  'p_nans',
                  'p_dist_val',
                  'scaled_max_val', 
                  'scaled_mean', 
                  'scaled_min_val',
                  'scaled_std_dev',
                  'castability',
                  'extractability',
                  'len_val']

y_columns = ['y_act']

X_train = data_train[x_columns].values
X_test = data_test[x_columns].values
y_train = data_train[y_columns].values.astype(int)
y_test = data_test[y_columns].values.astype(int)
# y_act = data_test['y_act'].values.astype(int)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6952, 15), (1739, 15), (6952, 1), (1739, 1))

In [18]:
sample_base = 'sample_'
# samples_train = []
# samples_test = []

i = 1
field_name = sample_base + str(i)
field_idx = x_columns.index(field_name)

samples1_train = X_train[:,field_idx].astype(str)
samples1_test = X_test[:,field_idx].astype(str)

In [19]:
field_name = 'Attribute_name'
atr_Xtrain = X_train[:,x_columns.index(field_name)].astype(str)
atr_Xtest = X_test[:,x_columns.index(field_name)].astype(str)

In [20]:
# define network parameters
max_features = 128
maxlen = 128

In [21]:
def generate_padded_tokens(train_list, test_list, maxlen=maxlen):
    tokenizer = keras_text.Tokenizer(char_level = True)
    tokenizer.fit_on_texts(train_list)
    tokenized_train = tokenizer.texts_to_sequences(train_list)
    pad_train = keras_seq.pad_sequences(tokenized_train, maxlen)

    tokenized_test = tokenizer.texts_to_sequences(test_list)
    pad_test = keras_seq.pad_sequences(tokenized_test, maxlen)
    return tokenizer, pad_train, pad_test

In [22]:
atr_tokenizer, atr_pad_train, atr_pad_test = generate_padded_tokens(atr_Xtrain, atr_Xtest)
sam_tokenizer, sam_pad_train, sam_pad_test = generate_padded_tokens(samples1_train, samples1_test)

In [23]:
X_train_left = X_train[:,6:]
X_test_left = X_test[:,6:]
X_train_left.shape, X_test_left.shape

((6952, 9), (1739, 9))

In [24]:
X_t = atr_pad_train
X_t1 = sam_pad_train
structured_data_train = X_train_left
X_te = atr_pad_test
X_te1 = sam_pad_test
structured_data_test = X_test_left
X_t.shape, X_t1.shape, structured_data_train.shape

((6952, 128), (6952, 128), (6952, 9))

In [25]:
import datetime
print('============================================================')
print('training starts at: \t', datetime.datetime.now())
print('============================================================\n')

training starts at: 	 2019-03-07 21:32:05.354997



In [26]:
X_t = atr_pad_train
X_t1 = sam_pad_train
structured_data_train = X_train_left
X_te = atr_pad_test
X_te1 = sam_pad_test
structured_data_test = X_test_left
X_t.shape, X_t1.shape, structured_data_train.shape

Xtrain = [X_t, X_t1, structured_data_train]
Xtest = [X_te, X_te1, structured_data_test]
Ytrain = to_categorical(y_train)
Ytest = y_test.flatten()
X_t.shape, X_te.shape, Ytrain.shape, Ytest.shape

((6952, 128), (1739, 128), (6952, 4), (1739,))

In [27]:
# http://sujitpal.blogspot.com/2018/02/using-snorkel-probabilistic-labels-for.html

def build_model(numfilters, embed_size, num_classes, tokenizer1, tokenizer2, maxlen):
    # name space input
    seq_input = Input(shape=(maxlen,))
    x = Embedding(input_dim = len(tokenizer1.word_counts)+1, output_dim = embed_size)(seq_input)
    x = Dropout(0.25)(x)
    x = Conv1D(numfilters*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv = [Dropout(0.5)(GlobalMaxPool1D()(x)), GlobalMaxPool1D()(x)]
    seq1 = concatenate(out_conv, axis = -1)  

    # sample space input
    seq_input2 = Input(shape=(maxlen, ))
    x = Embedding(input_dim = len(tokenizer2.word_counts)+1, output_dim = embed_size)(seq_input2)
    x = Dropout(0.25)(x)
    x = Conv1D(numfilters*2**(i), kernel_size = 3, activation = 'relu')(x)
    out_conv = [Dropout(0.5)(GlobalMaxPool1D()(x)), GlobalMaxPool1D()(x)]
    seq2 = concatenate(out_conv, axis = -1)

    # feature space input
    list_input = Input(shape=(9,))

    layersfin = keras.layers.concatenate([seq1, seq2, list_input])
    x = Dense(500, activation='relu')(layersfin)
    x = Dropout(0.25)(x)
    x = Dense(500, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    preds = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=[seq_input,seq_input2,list_input], outputs=[preds])
    return model

def compile_model(model):
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model

def fit_model(model, best_model_file, Xtrain, Ytrain, epochs=20, verbose=1):
    checkpoint = ModelCheckpoint(filepath=best_model_file,
                                 monitor='val_acc', 
                                 verbose=verbose,
                                 save_best_only=True)

    early = EarlyStopping(monitor="val_acc", patience=20)
    callbacks_list = [checkpoint, early] #early            

    history = model.fit(Xtrain, Ytrain, 
                        validation_split=0.1, 
                        epochs=epochs, 
                        batch_size=64,
                        verbose=verbose,
                        callbacks=callbacks_list)
    return history
  
def eval_report(title, Ytest, Ytest_, prob, verbose=True):
    if prob:
        ytest = np.argmax(Ytest, axis=1)
    else:
        ytest = Ytest
    ytest_ = Ytest_
    acc = accuracy_score(ytest, ytest_)
    cm = confusion_matrix(ytest, ytest_)
    if verbose:
        print("\n*** {:s}".format(title.upper()))
        print("accuracy: {:.3f}".format(acc*100))
        print("confusion matrix")
        print(cm)
    return acc

In [28]:
ne=32
md=128
epoch=50
num_classes = Ytrain.shape[1]

model_p = build_model(ne, md, num_classes, atr_tokenizer, sam_tokenizer, maxlen)
model_p = compile_model(model_p)

BEST_MODEL_P = 'best_weights'+str(ne)+str(md)+'.h5'

fit_model(model_p, BEST_MODEL_P, Xtrain, Ytrain, epoch, verbose=0)

best_model_p = load_model(BEST_MODEL_P)

In [29]:
Yptest_ = best_model_p.predict(Xtest)
Ytest_ = np.argmax(Yptest_,axis=1)
eval_report("Acc", Ytest, Ytest_, prob=False)


*** ACC
accuracy: 94.077
confusion matrix
[[408   7   8   8]
 [ 12 983   4   0]
 [ 12   3 100   4]
 [ 34   6   5 145]]


0.9407705577918344